In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import os
import re

import yaml
import h5py
import numpy as np
import pandas as pd
from scipy.io import wavfile
from IPython.display import display, Audio

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import magnolia.features.hdf5_iterator
from magnolia.features.data_preprocessing import make_stft_features
from magnolia.features.mixer import FeatureMixer
from magnolia.features.spectral_features import reconstruct

with open('some-data.yml') as f:
    some_data = yaml.safe_load(f)

In [3]:
# Set up paths and constants
data_dir = some_data['data_dir']
hdf5_dest = some_data['hdf5_dest']

dvds = [os.path.join(data_dir, 'DVD+R#{}'.format(i)) for i in range(1,4)]
rooms = [[os.path.join(dvd, room) for room in os.listdir(dvd) if re.search(r'.+Room[0-9]+', room) is not None] for dvd in dvds]
# flattening list, sorry
room_paths = [room for rooms_prime in rooms for room in rooms_prime]
wavs = { os.path.basename(room): sorted([os.path.join(room,wav) 
                                        for wav in os.listdir(room) if wav[-4:] == '.wav']) 
                                        for room in room_paths }
sources = {'LargeRoom08-0103':
          ['White noise',
'M1',
'M2',
'F1',
'F2',
'Music',
'M1',
'F1',
'M2',
'F2',
'White noise',
'White noise',
'White noise',
'White noise',
'M2',
'Three person conversation',
'M1',
'F1',
'F1',
'F2'],
      'MediumRoom07-1227': [
          'White noise',
'F1',
'F2',
'M1',
'M1',
'Four person conversation',
'Music',
'M2',
'F1',
'F2',
'White noise',
'White noise',
'M2',
'M1',
'White noise',
'White noise',
'F2',
'F1',
'M2',
'M1'
      ],
        'SmallRoom08-0104'  :
          ['Back Ground',
'White Noise',
'M1',
'M2',
'F1',
'F2',
'Music',
'M1',
'M1',
'White Noise',
'White Noise',
'F1',
'Source',
'F1',
'M2',
'White Noise',
'White Noise',
'F2',
'M2',
'M1',
'F2',
'F1']}

In [4]:
for room in wavs:
    for wav_path, source in zip(wavs[room], sources[room]):
        source = source.lower()
        spacetrans = str.maketrans(' ', '_')
        source = source.translate(spacetrans)
        print(source, ":", wav_path)
        filename = "{}_{}".format(
            source,
            os.path.basename(wav_path).lower()[:-4])
        with h5py.File(os.path.join(hdf5_dest, "{}.h5".format(filename)), 'w') as f:
            g = f.create_group(source)
            fs, wav = wavfile.read(wav_path)
            specs = [make_stft_features(wav[:,mic], fs, 10000.0) for mic in range(11)]
            specs = np.stack(specs,-1)
            g.create_dataset(filename, data=specs)

white_noise : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00001.wav


/opt/conda/envs/magnolia3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


f1 : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00002.wav
f2 : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00003.wav
m1 : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00004.wav
m1 : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00005.wav
four_person_conversation : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00006.wav
music : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00007.wav
m2 : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00008.wav
f1 : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00009.wav
f2 : /local_data/teams/magnolia/gfe-data/GFE Data/_2008_Audio_Set/DVD+R#2/MediumRoom07-1227/MED00010.wav
white_noise : /local_data/team

In [ ]:
hdf5_files = [hdf5_path for hdf5_path in os.listdir(hdf5_dest) if 'white_noise' not in hdf5_path]
test_file_1 = os.path.join(hdf5_dest,np.random.choice(hdf5_files))
test_file_2 = os.path.join(hdf5_dest,np.random.choice(hdf5_files))

In [19]:
fmix = FeatureMixer((test_file_1, test_file_2), shape=(100,None))
mix, wav1, wav2 = next(fmix)
mix_t = reconstruct(mix[:,:,2], mix[:,:,2], 10000, 0.05, 0.025)
display(Audio(mix_t, rate = 10000))

In [20]:
wav1.shape

(100, 257, 11)

In [21]:
wav1_t = reconstruct(wav1[:,:,10], wav1[:,:,10], 10000, 0.05, 0.025)
display(Audio(wav1_t, rate = 10000))
wav2_t = reconstruct(wav2[:,:,10], wav2[:,:,10], 10000, 0.05, 0.025)
display(Audio(wav2_t, rate = 10000))

In [22]:
wav2_t = reconstruct(wav2[:,:,10], wav2[:,:,10], 10000, 0.05, 0.025)
display(Audio(wav2_t, rate = 10000))

In [23]:
print(test_file_1)
print(test_file_2)

/local_data/teams/magnolia/gfe-data/hdf5/f1_small012.h5
/local_data/teams/magnolia/gfe-data/hdf5/music_med00007.h5
